In [25]:
import pandas as pd


In [26]:
filelist = ['birth.csv', 'breed.csv', 'report.csv', 'spec.csv','submission.csv']

# Read File

In [27]:
birth = pd.read_csv('./data/' + filelist[0])
breed = pd.read_csv('./data/' + filelist[1])
report = pd.read_csv('./data/' + filelist[2])
spec = pd.read_csv('./data/' + filelist[3])
submission = pd.read_csv('./data/' + filelist[4])
submission

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,1
0,480,NaN
1,742,NaN
2,743,NaN
3,744,NaN
4,745,NaN
...,...,...
4258,37481,NaN
4259,37482,NaN
4260,37483,NaN
4261,37484,NaN


In [28]:
birth_index = ['乳牛編號', '分娩日期', '乾乳日期','犢牛編號1','犢牛編號2','母牛體重','登錄日期','計算胎次','胎次','分娩難易度','犢牛體型','犢牛性別','酪農場代號']
report_index = ['ID','資料年度','資料月份','酪農場代號','乳牛編號','父親精液編號','母親乳牛編號','出生日期','胎次','泌乳天數','乳量','最近分娩日期','採樣日期','月齡','檢測日期','最後配種日期','最後配種精液','配種次數','前次分娩日期','第一次配種日期','第一次配種精液']
breed_index = ['乳牛編號','配種日期','配種精液','登錄日期','孕檢','配種方式','精液種類','酪農場代號']
spec_index = ['乳牛編號','狀況類別','狀況代號','狀況日期','備註','登錄日期','酪農場代號']
birth.columns = birth_index
report.columns = report_index
breed.columns = breed_index
spec.columns = spec_index
# birth.fillna(0,inplace = True)
# report.fillna(0,inplace = True)
# breed.fillna(0, inplace = True)
# spec.fillna(0, inplace = True)
breed

,乳牛編號,配種日期,配種精液,登錄日期,孕檢,配種方式,精液種類,酪農場代號
0,11361,2005/9/2 00:00,1H5506,2005/9/28 00:00,0,NaN,NaN,A
1,11469,2000/1/17 00:00,72H896,2000/1/31 00:00,0,NaN,NaN,A
2,11930,2000/5/12 00:00,9H2164,2000/5/22 00:00,0,NaN,NaN,A
3,11930,2000/5/26 00:00,1H2993,2000/7/6 00:00,0,NaN,NaN,A
4,11930,2000/7/4 00:00,14H2403,1988/7/29 00:00,0,NaN,NaN,A
...,...,...,...,...,...,...,...,...
21045,6187429,2019/7/17 00:00,001HO12901,2019/7/26 00:00,0,NaN,NaN,C
21046,6187431,2019/7/24 00:00,001HO12901,2019/7/26 00:00,0,NaN,NaN,C
21047,6187441,2019/7/14 00:00,001HO12901,2019/7/26 00:00,0,NaN,NaN,C
21048,6187442,2019/7/17 00:00,001HO12901,2019/7/26 00:00,0,NaN,NaN,C


In [29]:
# one hot encode report
tx = pd.get_dummies(report['酪農場代號'], prefix = '酪農場代號')
report = pd.concat([report, tx], axis = 1)

In [30]:
Train_ID = [rep_val for rep_val in report['ID'].values if rep_val not in submission['ID'].values]
Train_report = report.loc[report['ID'].isin(Train_ID)]
Train_Number = Train_report['乳牛編號'].values
Train_report['ID'].values

array([    1,     2,     3, ..., 37515, 37516, 37517], dtype=int64)

In [31]:
Test_ID = [rep_val for rep_val in report['ID'].values if rep_val in submission['ID'].values]
Test_report = report.loc[report['ID'].isin(Test_ID)]
Test_Number = Test_report['乳牛編號'].values
Test_report

,ID,資料年度,資料月份,酪農場代號,乳牛編號,父親精液編號,母親乳牛編號,出生日期,胎次,泌乳天數,...,檢測日期,最後配種日期,最後配種精液,配種次數,前次分娩日期,第一次配種日期,第一次配種精液,酪農場代號_A,酪農場代號_B,酪農場代號_C
479,480,2019,1,A,52631,501H9952,97040152,2011/3/17 00:00,4,435.0,...,2019/1/15 00:00,2019/2/3 00:00,551HO00657,6,2016/8/4 00:00,2018/1/6 00:00,614HO07276,1,0,0
741,742,2019,2,A,52640,501H9952,96040567,2011/4/1 00:00,4,13.0,...,2019/2/12 00:00,2019/2/28 00:00,551HO00695,0,2017/6/17 00:00,2019/2/28 00:00,551HO00695,1,0,0
742,743,2019,3,A,52640,501H9952,96040567,2011/4/1 00:00,4,35.0,...,2019/3/6 00:00,2019/2/28 00:00,551HO00695,1,2017/6/17 00:00,2019/2/28 00:00,551HO00695,1,0,0
743,744,2019,4,A,52640,501H9952,96040567,2011/4/1 00:00,4,70.0,...,2019/4/11 00:00,2019/2/28 00:00,551HO00695,1,2017/6/17 00:00,2019/2/28 00:00,551HO00695,1,0,0
744,745,2019,5,A,52640,501H9952,96040567,2011/4/1 00:00,4,98.0,...,2019/5/9 00:00,2019/2/28 00:00,551HO00695,1,2017/6/17 00:00,2019/2/28 00:00,551HO00695,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37480,37481,2019,6,C,6187446,001HO11511,4.17046e+06,2017/6/1 00:00,1,15.0,...,2019/6/27 00:00,2019/7/24 00:00,001HO12901,0,NaN,2019/7/24 00:00,001HO12901,0,0,1
37481,37482,2019,6,C,6187448,001HO11511,3.17231e+06,2017/6/17 00:00,1,22.0,...,2019/6/27 00:00,NaN,NaN,0,NaN,NaN,NaN,0,0,1
37482,37483,2019,6,C,6187449,001HO11511,2.17684e+06,2017/8/10 00:00,1,21.0,...,2019/6/27 00:00,NaN,NaN,0,NaN,NaN,NaN,0,0,1
37483,37484,2019,7,C,6187449,001HO11511,2.17684e+06,2017/8/10 00:00,1,51.0,...,2019/7/26 00:00,NaN,NaN,0,NaN,NaN,NaN,0,0,1


In [32]:
report_drop_columns = ['ID','資料年度','資料月份','父親精液編號','母親乳牛編號','出生日期','最近分娩日期','採樣日期','檢測日期'
               ,'最後配種日期','最後配種精液','前次分娩日期','第一次配種日期','第一次配種精液', '酪農場代號']
Test_report_drop_columns = ['資料年度','資料月份','父親精液編號','母親乳牛編號','出生日期','最近分娩日期','採樣日期','檢測日期'
               ,'最後配種日期','最後配種精液','前次分娩日期','第一次配種日期','第一次配種精液', '酪農場代號']
Train_report_x = Train_report.drop(columns=report_drop_columns, axis = 1)

Test_report_x = Test_report.drop(columns=Test_report_drop_columns, axis = 1)
Test_report_x

,ID,乳牛編號,胎次,泌乳天數,乳量,月齡,配種次數,酪農場代號_A,酪農場代號_B,酪農場代號_C
479,480,52631,4,435.0,NaN,94,6,1,0,0
741,742,52640,4,13.0,NaN,94,0,1,0,0
742,743,52640,4,35.0,NaN,95,1,1,0,0
743,744,52640,4,70.0,NaN,96,1,1,0,0
744,745,52640,4,98.0,NaN,97,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
37480,37481,6187446,1,15.0,NaN,24,0,0,0,1
37481,37482,6187448,1,22.0,NaN,24,0,0,0,1
37482,37483,6187449,1,21.0,NaN,22,0,0,0,1
37483,37484,6187449,1,51.0,NaN,23,0,0,0,1


In [33]:
# # breed: one hot encode 
# tx = pd.get_dummies(breed['配種方式'], prefix = '配種方式')
# breed = pd.concat([breed, tx], axis = 1)
# tx = pd.get_dummies(breed['精液種類'], prefix = '精液種類')
# breed = pd.concat([breed, tx], axis = 1)

# breed_drop_columns = ['配種日期','配種精液','登錄日期','孕檢', '配種方式', '精液種類', '酪農場代號']
# breed_x = breed.drop(columns=breed_drop_columns, axis = 1)

# Train_breed_x = breed_x.loc[breed_x['乳牛編號'].isin(Train_Number)]
# Test_breed_x = breed_x.loc[breed_x['乳牛編號'].isin(Test_Number)]
# Test_breed_x

In [34]:
# birth feature
birth_drop_columns = ['分娩日期', '乾乳日期', '犢牛編號1', '犢牛編號2', '登錄日期', '計算胎次', '犢牛體型', '犢牛性別', '酪農場代號']
birth_x = birth.drop(columns=birth_drop_columns, axis = 1)

Train_birth_x = birth_x.loc[birth_x['乳牛編號'].isin(Train_Number)]
Test_birth_x = birth_x.loc[birth_x['乳牛編號'].isin(Test_Number)]
Train_birth_x

,乳牛編號,母牛體重,胎次,分娩難易度
0,87121677,NaN,2,1.0
1,96126940,522.0,2,2.0
2,95181708,670.0,5,1.0
3,97127665,NaN,3,2.0
4,97182634,630.0,4,1.0
...,...,...,...,...
3755,4181793,590.0,2,3.0
3756,5184341,560.0,2,1.0
3757,5184348,580.0,2,1.0
3758,97182634,730.0,9,1.0


In [35]:
# def season(time):
#     if len(item) == 0:
#         return 0
#     temp = int(item.split('/')[1])
#     if temp >= 1 and temp <= 3:
#         return '春'
#     elif temp >= 4 and temp <= 6:
#         return '夏'
#     elif temp >= 7 and temp <= 9:
#         return '秋'
#     elif temp >= 10 and temp <= 12:
#         return '冬'
#     else:
#         return 0

In [36]:
# for i, item in enumerate(birth['分娩日期']):
#     birth.loc[i, '分娩日期'] = season(item)
# birth

In [37]:
# res = pd.merge(birth, breed, on = '乳牛編號')
# res = pd.merge(res, report, on = '乳牛編號')
# res = pd.merge(res, spec, on = '乳牛編號')
# res['分娩日期'][1]

# Processing Na

In [38]:
days = Train_report_x['泌乳天數'].mean()
Train_report_x['泌乳天數'].fillna(days, inplace = True)

weight = Train_birth_x['母牛體重'].mean()
Train_birth_x['母牛體重'].fillna(weight, inplace = True)

weight = Test_birth_x['母牛體重'].mean()
Test_birth_x['母牛體重'].fillna(weight, inplace = True)

hard = Train_birth_x['分娩難易度'].mean()
Train_birth_x['分娩難易度'].fillna(hard, inplace = True)

hard = Train_birth_x['分娩難易度'].mean()
Test_birth_x['分娩難易度'].fillna(hard, inplace = True)

D:\anaconda3\lib\site-packages\pandas\core\series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


# Data Normalization

In [39]:
birth_normal = ['母牛體重', '胎次', '分娩難易度']
for normal_col in birth_normal:
    normalized_x = (Train_birth_x[normal_col]-Train_birth_x[normal_col].mean()) / Train_birth_x[normal_col].std()
    Train_birth_x[normal_col] = normalized_x
    
    normalized_x = (Test_birth_x[normal_col]-Test_birth_x[normal_col].mean()) / Test_birth_x[normal_col].std()
    Test_birth_x[normal_col] = normalized_x
    
report_normal = ['泌乳天數', '胎次', '月齡', '配種次數']
for normal_col in report_normal:
    normalized_x = (Train_report_x[normal_col]-Train_report_x[normal_col].mean()) / Train_report_x[normal_col].std()
    Train_report_x[normal_col] = normalized_x
    
    normalized_x = (Test_report_x[normal_col]-Test_report_x[normal_col].mean()) / Test_report_x[normal_col].std()
    Test_report_x[normal_col] = normalized_x

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


# Merge Data

In [40]:
# x = pd.merge(Train_report_x, Train_breed_x, on = ['乳牛編號'])
x = pd.merge(Train_report_x, Train_birth_x, on= ['乳牛編號'])
y = x['乳量'].values
x = x.drop(columns = ['乳牛編號','乳量'], axis = 1)
x

,胎次_x,泌乳天數,月齡,配種次數,酪農場代號_A,酪農場代號_B,酪農場代號_C,母牛體重,胎次_y,分娩難易度
0,-0.778490,-0.853648,-1.115650,-0.454949,1,0,0,5.065279e-15,-0.868055,0.000000
1,-0.778490,-0.853648,-1.115650,-0.454949,1,0,0,5.065279e-15,-0.086893,0.000000
2,-0.778490,-0.660747,-1.063834,-0.454949,1,0,0,5.065279e-15,-0.868055,0.000000
3,-0.778490,-0.660747,-1.063834,-0.454949,1,0,0,5.065279e-15,-0.086893,0.000000
4,-0.778490,-0.441240,-1.012018,0.014484,1,0,0,5.065279e-15,-0.868055,0.000000
...,...,...,...,...,...,...,...,...,...,...
87181,4.941512,-1.292663,4.273228,-0.924381,0,0,1,6.445178e+00,4.600083,-0.606436
87182,4.941512,-0.461195,4.480492,-0.924381,0,0,1,6.445178e+00,4.600083,-0.606436
87183,4.941512,-0.254991,4.532309,-0.924381,0,0,1,6.445178e+00,4.600083,-0.606436
87184,4.941512,-0.062090,4.584125,-0.924381,0,0,1,6.445178e+00,4.600083,-0.606436


In [43]:
# test = pd.merge(Test_report_x, Test_breed_x, on =  ['乳牛編號'], how = 'left')
test = pd.merge(Test_report_x, Test_birth_x, on =  ['乳牛編號'], how = 'left')

test_without_ID = test.drop(columns = ['乳牛編號','乳量','ID'], axis = 1)
test_without_ID

,胎次_x,泌乳天數,月齡,配種次數,酪農場代號_A,酪農場代號_B,酪農場代號_C,母牛體重,胎次_y,分娩難易度
0,1.636775,1.671192,2.487608,1.995674,1,0,0,5.170364e-15,-0.765127,0.051669
1,1.636775,1.671192,2.487608,1.995674,1,0,0,5.170364e-15,0.076127,0.051669
2,1.636775,1.671192,2.487608,1.995674,1,0,0,5.170364e-15,0.917382,0.051669
3,1.636775,1.671192,2.487608,1.995674,1,0,0,5.170364e-15,1.758636,0.051669
4,1.636775,-1.251356,2.487608,-0.924708,1,0,0,5.170364e-15,-0.765127,0.051669
...,...,...,...,...,...,...,...,...,...,...
8825,-0.835203,-1.237505,-1.211469,-0.924708,0,0,1,-1.193437e+00,-0.765127,-0.578055
8826,-0.835203,-1.189027,-1.211469,-0.924708,0,0,1,-3.467387e+00,-0.765127,-0.578055
8827,-0.835203,-1.195952,-1.317157,-0.924708,0,0,1,-1.193437e+00,-0.765127,-0.578055
8828,-0.835203,-0.988188,-1.264313,-0.924708,0,0,1,-1.193437e+00,-0.765127,-0.578055
